In [4]:
import subprocess
import sys

# Dictionary mapping module names to their pip install package names
required_packages = {
    'numpy': 'numpy',
    'tensorflow': 'tensorflow',
    'cv2': 'opencv-python',       # 'cv2' comes from opencv-python
    'pywt': 'PyWavelets',         # 'pywt' comes from PyWavelets
    'matplotlib': 'matplotlib',
    'scipy': 'scipy',
    'sklearn': 'scikit-learn',    # scikit-learn provides the 'sklearn' module
    'pandas': 'pandas'
}

def install_package(package):
    """Installs a package using pip."""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check for each package and install if missing
for module_name, package_name in required_packages.items():
    try:
        __import__(module_name)
        print(f"{module_name} is already installed.")
    except ImportError:
        print(f"{module_name} is not installed. Installing {package_name}...")
        install_package(package_name)

print("All required packages are installed.")


numpy is already installed.
tensorflow is already installed.
cv2 is already installed.
pywt is already installed.
matplotlib is already installed.
scipy is already installed.
sklearn is already installed.
pandas is already installed.
All required packages are installed.


# Simple CNN - (MNIST, Fashion MNIST)

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import pywt
import logging
import matplotlib.pyplot as plt
from scipy.fftpack import fft2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize grayscale images and reshape
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
X_train = np.expand_dims(X_train, axis=-1)  # Shape: (num_samples, 28, 28, 1)
X_test = np.expand_dims(X_test, axis=-1)
logging.info("Images preprocessed.")

# ------------- 1. Compute FFT Features (Real + Imaginary as 2 Channels) -------------
def compute_fft_image(images):
    fft_images = []
    for img in images:
        fft_img = fft2(img[..., 0])  # FFT on the single grayscale channel
        fft_real = abs(np.real(fft_img))  # Magnitude part
        fft_imag = np.angle(np.imag(fft_img))  # Phase part
        fft_image = np.stack([fft_real, fft_imag], axis=-1)  # Shape: (28, 28, 2)
        fft_images.append(fft_image)
    return np.array(fft_images)

# ------------- 2. Compute Wavelet Features (4 Channels) -------------
def compute_wavelet_image(images, wavelet='haar', level=1):
    wavelet_images = np.empty((images.shape[0], 14, 14, 4), dtype=np.float16)  # Pre-allocate
    for i, img in enumerate(images):
        coeffs2 = pywt.wavedec2(img[..., 0], wavelet, level=level)  # Wavelet on single channel
        cA, (cH, cV, cD) = coeffs2  # Extract coefficients
        cA, cH, cV, cD = [cv2.resize(c, (14, 14), interpolation=cv2.INTER_LINEAR) for c in [cA, cH, cV, cD]]
        wavelet_images[i] = np.stack([cA, cH, cV, cD], axis=-1)
    return wavelet_images

# ------------- 3. Define CNN Model -------------
def build_cnn(input_shape, num_classes=10):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

# Model Evaluation Function
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test).argmax(axis=1)
    accuracy = np.mean(y_pred == y_test)
    logging.info(f"{model_name} Test Accuracy: {accuracy:.4f}")
    print(f"=== Classification Report for {model_name} ===")
    print(classification_report(y_test, y_pred))
    print(f"=== Confusion Matrix for {model_name} ===")
    print(confusion_matrix(y_test, y_pred))

# Train CNN on Original MNIST
cnn_mnist = build_cnn(input_shape=(28, 28, 1))
cnn_mnist.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_mnist.save("cnn_mnist_model.keras")
evaluate_model(cnn_mnist, X_test, y_test, "Original MNIST")

# Train CNN on FFT Transformed Images
X_train_fft = compute_fft_image(X_train)
X_test_fft = compute_fft_image(X_test)
cnn_fft = build_cnn(input_shape=(28, 28, 2))
cnn_fft.fit(X_train_fft, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_fft.save("cnn_fft_mnist_model.keras")
evaluate_model(cnn_fft, X_test_fft, y_test, "FFT MNIST")

# Train CNN on Wavelet Transformed Images
X_train_wavelet = compute_wavelet_image(X_train)
X_test_wavelet = compute_wavelet_image(X_test)
cnn_wavelet = build_cnn(input_shape=(14, 14, 4))
cnn_wavelet.fit(X_train_wavelet, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_wavelet.save("cnn_wavelet_mnist_model.keras")
evaluate_model(cnn_wavelet, X_test_wavelet, y_test, "Wavelet MNIST")


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import pywt
import logging
import matplotlib.pyplot as plt
from scipy.fftpack import fft2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize grayscale images and reshape
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
X_train = np.expand_dims(X_train, axis=-1)  # Shape: (num_samples, 28, 28, 1)
X_test = np.expand_dims(X_test, axis=-1)
logging.info("Images preprocessed.")

# ------------- 1. Compute FFT Features (Real + Imaginary as 2 Channels) -------------
def compute_fft_image(images):
    fft_images = []
    for img in images:
        fft_img = fft2(img[..., 0])  # FFT on the single grayscale channel
        fft_real = abs(np.real(fft_img))  # Magnitude part
        fft_imag = np.angle(np.imag(fft_img))  # Phase part
        fft_image = np.stack([fft_real, fft_imag], axis=-1)  # Shape: (28, 28, 2)
        fft_images.append(fft_image)
    return np.array(fft_images)

# ------------- 2. Compute Wavelet Features (4 Channels) -------------
def compute_wavelet_image(images, wavelet='haar', level=1):
    wavelet_images = np.empty((images.shape[0], 14, 14, 4), dtype=np.float16)  # Pre-allocate
    for i, img in enumerate(images):
        coeffs2 = pywt.wavedec2(img[..., 0], wavelet, level=level)  # Wavelet on single channel
        cA, (cH, cV, cD) = coeffs2  # Extract coefficients
        cA, cH, cV, cD = [cv2.resize(c, (14, 14), interpolation=cv2.INTER_LINEAR) for c in [cA, cH, cV, cD]]
        wavelet_images[i] = np.stack([cA, cH, cV, cD], axis=-1)
    return wavelet_images

# ------------- 3. Define CNN Model -------------
def build_cnn(input_shape, num_classes=10):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ModelCheckpoint('fashion_mnist_best_model.keras', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

# Model Evaluation Function
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test).argmax(axis=1)
    accuracy = np.mean(y_pred == y_test)
    logging.info(f"{model_name} Test Accuracy: {accuracy:.4f}")
    print(f"=== Classification Report for {model_name} ===")
    print(classification_report(y_test, y_pred))
    print(f"=== Confusion Matrix for {model_name} ===")
    print(confusion_matrix(y_test, y_pred))

# Train CNN on Original MNIST
cnn_mnist = build_cnn(input_shape=(28, 28, 1))
cnn_mnist.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_mnist.save("cnn_fashion_mnist_model.keras")
evaluate_model(cnn_mnist, X_test, y_test, "Original MNIST")

# Train CNN on FFT Transformed Images
X_train_fft = compute_fft_image(X_train)
X_test_fft = compute_fft_image(X_test)
cnn_fft = build_cnn(input_shape=(28, 28, 2))
cnn_fft.fit(X_train_fft, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_fft.save("cnn_fft_fashion_mnist_model.keras")
evaluate_model(cnn_fft, X_test_fft, y_test, "FFT MNIST")

# Train CNN on Wavelet Transformed Images
X_train_wavelet = compute_wavelet_image(X_train)
X_test_wavelet = compute_wavelet_image(X_test)
cnn_wavelet = build_cnn(input_shape=(14, 14, 4))
cnn_wavelet.fit(X_train_wavelet, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_wavelet.save("cnn_wavelet_fashion_mnist_model.keras")
evaluate_model(cnn_wavelet, X_test_wavelet, y_test, "Wavelet MNIST")


# VGG16 Model - (MNIST, Fashion MNIST)

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import pywt
import logging
import matplotlib.pyplot as plt
from scipy.fftpack import fft2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Import VGG16 and its preprocessing function
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize grayscale images and reshape
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
X_train = np.expand_dims(X_train, axis=-1)  # Shape: (num_samples, 28, 28, 1)
X_test = np.expand_dims(X_test, axis=-1)
logging.info("Images preprocessed.")

# ------------- 1. Compute FFT Features (Real + Imaginary as 2 Channels) -------------
def compute_fft_image(images):
    fft_images = []
    for img in images:
        fft_img = fft2(img[..., 0])  # FFT on the single grayscale channel
        fft_real = abs(np.real(fft_img))  # Magnitude part
        fft_imag = np.angle(np.imag(fft_img))  # Phase part
        fft_image = np.stack([fft_real, fft_imag], axis=-1)  # Shape: (28, 28, 2)
        fft_images.append(fft_image)
    return np.array(fft_images)

# ------------- 2. Compute Wavelet Features (4 Channels) -------------
def compute_wavelet_image(images, wavelet='haar', level=1):
    wavelet_images = np.empty((images.shape[0], 14, 14, 4), dtype=np.float16)  # Pre-allocate
    for i, img in enumerate(images):
        coeffs2 = pywt.wavedec2(img[..., 0], wavelet, level=level)  # Wavelet on single channel
        cA, (cH, cV, cD) = coeffs2  # Extract coefficients
        cA, cH, cV, cD = [cv2.resize(c, (14, 14), interpolation=cv2.INTER_LINEAR) for c in [cA, cH, cV, cD]]
        wavelet_images[i] = np.stack([cA, cH, cV, cD], axis=-1)
    return wavelet_images

# ------------- 3. Define CNN Model -------------
# -------------------- Input Transformer --------------------
def build_input_transformer(input_shape, target_shape=(32, 32, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Resizing(target_shape[0], target_shape[1])(inputs)
    if input_shape[-1] != target_shape[-1]:
        x = Conv2D(target_shape[-1], (3, 3), padding='same', activation='relu')(x)
    model = tf.keras.Model(inputs=inputs, outputs=x, name='input_transformer')
    return model
    
def build_vgg16_model(input_shape, num_classes=10, target_shape=(32, 32, 3)):
    transformer = build_input_transformer(input_shape, target_shape)
    inputs = transformer.input
    x = transformer.output
    x = tf.keras.layers.Lambda(lambda img: preprocess_input(img * 255.0))(x)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=target_shape)
    base_model.trainable = False  # Freeze VGG16 layers
    x = base_model(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='vgg16_model')
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ModelCheckpoint('vgg16_best_mnist_model.keras', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

# Model Evaluation Function
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test).argmax(axis=1)
    accuracy = np.mean(y_pred == y_test)
    logging.info(f"{model_name} Test Accuracy: {accuracy:.4f}")
    print(f"=== Classification Report for {model_name} ===")
    print(classification_report(y_test, y_pred))
    print(f"=== Confusion Matrix for {model_name} ===")
    print(confusion_matrix(y_test, y_pred))

# Train CNN on Original MNIST
cnn_mnist = build_vgg16_model(input_shape=(28, 28, 1))
cnn_mnist.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_mnist.save("vgg16_mnist_model.keras")
evaluate_model(cnn_mnist, X_test, y_test, "vgg16 Original MNIST")

# Train CNN on FFT Transformed Images
X_train_fft = compute_fft_image(X_train)
X_test_fft = compute_fft_image(X_test)
cnn_fft = build_vgg16_model(input_shape=(28, 28, 2))
cnn_fft.fit(X_train_fft, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_fft.save("vgg16_fft_mnist_model.keras")
evaluate_model(cnn_fft, X_test_fft, y_test, "vgg16 FFT MNIST")

# Train CNN on Wavelet Transformed Images
X_train_wavelet = compute_wavelet_image(X_train)
X_test_wavelet = compute_wavelet_image(X_test)
cnn_wavelet = build_vgg16_model(input_shape=(14, 14, 4))
cnn_wavelet.fit(X_train_wavelet, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_wavelet.save("vgg16_wavelet_mnist_model.keras")
evaluate_model(cnn_wavelet, X_test_wavelet, y_test, "vgg16 Wavelet MNIST")


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import pywt
import logging
import matplotlib.pyplot as plt
from scipy.fftpack import fft2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd


# Import VGG16 and its preprocessing function
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize grayscale images and reshape
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
X_train = np.expand_dims(X_train, axis=-1)  # Shape: (num_samples, 28, 28, 1)
X_test = np.expand_dims(X_test, axis=-1)
logging.info("Images preprocessed.")

# ------------- 1. Compute FFT Features (Real + Imaginary as 2 Channels) -------------
def compute_fft_image(images):
    fft_images = []
    for img in images:
        fft_img = fft2(img[..., 0])  # FFT on the single grayscale channel
        fft_real = abs(np.real(fft_img))  # Magnitude part
        fft_imag = np.angle(np.imag(fft_img))  # Phase part
        fft_image = np.stack([fft_real, fft_imag], axis=-1)  # Shape: (28, 28, 2)
        fft_images.append(fft_image)
    return np.array(fft_images)

# ------------- 2. Compute Wavelet Features (4 Channels) -------------
def compute_wavelet_image(images, wavelet='haar', level=1):
    wavelet_images = np.empty((images.shape[0], 14, 14, 4), dtype=np.float16)  # Pre-allocate
    for i, img in enumerate(images):
        coeffs2 = pywt.wavedec2(img[..., 0], wavelet, level=level)  # Wavelet on single channel
        cA, (cH, cV, cD) = coeffs2  # Extract coefficients
        cA, cH, cV, cD = [cv2.resize(c, (14, 14), interpolation=cv2.INTER_LINEAR) for c in [cA, cH, cV, cD]]
        wavelet_images[i] = np.stack([cA, cH, cV, cD], axis=-1)
    return wavelet_images

# ------------- 3. Define CNN Model -------------
def build_input_transformer(input_shape, target_shape=(32, 32, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Resizing(target_shape[0], target_shape[1])(inputs)
    if input_shape[-1] != target_shape[-1]:
        x = Conv2D(target_shape[-1], (3, 3), padding='same', activation='relu')(x)
    model = tf.keras.Model(inputs=inputs, outputs=x, name='input_transformer')
    return model
    
def build_vgg16_model(input_shape, num_classes=10, target_shape=(32, 32, 3)):
    transformer = build_input_transformer(input_shape, target_shape)
    inputs = transformer.input
    x = transformer.output
    x = tf.keras.layers.Lambda(lambda img: preprocess_input(img * 255.0))(x)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=target_shape)
    base_model.trainable = False  # Freeze VGG16 layers
    x = base_model(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='vgg16_model')
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ModelCheckpoint('vgg16_fashion_mnist_best_model.keras', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

# Model Evaluation Function
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test).argmax(axis=1)
    accuracy = np.mean(y_pred == y_test)
    logging.info(f"{model_name} Test Accuracy: {accuracy:.4f}")
    print(f"=== Classification Report for {model_name} ===")
    print(classification_report(y_test, y_pred))
    print(f"=== Confusion Matrix for {model_name} ===")
    print(confusion_matrix(y_test, y_pred))

# Train CNN on Original MNIST
cnn_mnist = build_vgg16_model(input_shape=(28, 28, 1))
cnn_mnist.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_mnist.save("vgg16_fashion_mnist_model.keras")
evaluate_model(cnn_mnist, X_test, y_test, "vgg16 Original MNIST")

# Train CNN on FFT Transformed Images
X_train_fft = compute_fft_image(X_train)
X_test_fft = compute_fft_image(X_test)
cnn_fft = build_vgg16_model(input_shape=(28, 28, 2))
cnn_fft.fit(X_train_fft, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_fft.save("vgg16_fft_fashion_mnist_model.keras")
evaluate_model(cnn_fft, X_test_fft, y_test, "vgg16 FFT MNIST")

# Train CNN on Wavelet Transformed Images
X_train_wavelet = compute_wavelet_image(X_train)
X_test_wavelet = compute_wavelet_image(X_test)
cnn_wavelet = build_vgg16_model(input_shape=(14, 14, 4))
cnn_wavelet.fit(X_train_wavelet, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_wavelet.save("vgg16_wavelet_fashion_mnist_model.keras")
evaluate_model(cnn_wavelet, X_test_wavelet, y_test, "vgg16 Wavelet MNIST")


# Resnet50 Model - (MNIST, Fashion MNIST)

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import pywt
import logging
import matplotlib.pyplot as plt
from scipy.fftpack import fft2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input


# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize grayscale images and reshape
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
X_train = np.expand_dims(X_train, axis=-1)  # Shape: (num_samples, 28, 28, 1)
X_test = np.expand_dims(X_test, axis=-1)
logging.info("Images preprocessed.")

# ------------- 1. Compute FFT Features (Real + Imaginary as 2 Channels) -------------
def compute_fft_image(images):
    fft_images = []
    for img in images:
        fft_img = fft2(img[..., 0])  # FFT on the single grayscale channel
        fft_real = abs(np.real(fft_img))  # Magnitude part
        fft_imag = np.angle(np.imag(fft_img))  # Phase part
        fft_image = np.stack([fft_real, fft_imag], axis=-1)  # Shape: (28, 28, 2)
        fft_images.append(fft_image)
    return np.array(fft_images)

# ------------- 2. Compute Wavelet Features (4 Channels) -------------
def compute_wavelet_image(images, wavelet='haar', level=1):
    wavelet_images = np.empty((images.shape[0], 14, 14, 4), dtype=np.float16)  # Pre-allocate
    for i, img in enumerate(images):
        coeffs2 = pywt.wavedec2(img[..., 0], wavelet, level=level)  # Wavelet on single channel
        cA, (cH, cV, cD) = coeffs2  # Extract coefficients
        cA, cH, cV, cD = [cv2.resize(c, (14, 14), interpolation=cv2.INTER_LINEAR) for c in [cA, cH, cV, cD]]
        wavelet_images[i] = np.stack([cA, cH, cV, cD], axis=-1)
    return wavelet_images

# ------------- 3. Define CNN Model -------------
# -------------------- Input Transformer --------------------
def build_input_transformer(input_shape, target_shape=(224, 224, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Resizing(target_shape[0], target_shape[1])(inputs)
    if input_shape[-1] != target_shape[-1]:
        x = Conv2D(target_shape[-1], (3, 3), padding='same', activation='relu')(x)
    model = tf.keras.Model(inputs=inputs, outputs=x, name='input_transformer')
    return model
    
def build_resnet_model(input_shape, num_classes=10, target_shape=(224, 224, 3)):
    transformer = build_input_transformer(input_shape, target_shape)
    inputs = transformer.input
    x = transformer.output
    x = tf.keras.layers.Lambda(lambda img: preprocess_input(img * 255.0))(x)
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=target_shape)
    base_model.trainable = False  # Freeze ResNet layers
    x = base_model(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='resnet_model')
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ModelCheckpoint('resnet_best_mnist_model.keras', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

# Model Evaluation Function
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test).argmax(axis=1)
    accuracy = np.mean(y_pred == y_test)
    logging.info(f"{model_name} Test Accuracy: {accuracy:.4f}")
    print(f"=== Classification Report for {model_name} ===")
    print(classification_report(y_test, y_pred))
    print(f"=== Confusion Matrix for {model_name} ===")
    print(confusion_matrix(y_test, y_pred))

# Train CNN on Original MNIST
cnn_mnist = build_resnet_model(input_shape=(28, 28, 1))
cnn_mnist.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_mnist.save("resnet_mnist_model.keras")
evaluate_model(cnn_mnist, X_test, y_test, "resnet Original MNIST")

# Train CNN on FFT Transformed Images
X_train_fft = compute_fft_image(X_train)
X_test_fft = compute_fft_image(X_test)
cnn_fft = build_resnet_model(input_shape=(28, 28, 2))
cnn_fft.fit(X_train_fft, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_fft.save("resnet_fft_mnist_model.keras")
evaluate_model(cnn_fft, X_test_fft, y_test, "resnet FFT MNIST")

# Train CNN on Wavelet Transformed Images
X_train_wavelet = compute_wavelet_image(X_train)
X_test_wavelet = compute_wavelet_image(X_test)
cnn_wavelet = build_resnet_model(input_shape=(14, 14, 4))
cnn_wavelet.fit(X_train_wavelet, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_wavelet.save("resnet_wavelet_mnist_model.keras")
evaluate_model(cnn_wavelet, X_test_wavelet, y_test, "resnet Wavelet MNIST")


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import pywt
import logging
import matplotlib.pyplot as plt
from scipy.fftpack import fft2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize grayscale images and reshape
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
X_train = np.expand_dims(X_train, axis=-1)  # Shape: (num_samples, 28, 28, 1)
X_test = np.expand_dims(X_test, axis=-1)
logging.info("Images preprocessed.")

# ------------- 1. Compute FFT Features (Real + Imaginary as 2 Channels) -------------
def compute_fft_image(images):
    fft_images = []
    for img in images:
        fft_img = fft2(img[..., 0])  # FFT on the single grayscale channel
        fft_real = abs(np.real(fft_img))  # Magnitude part
        fft_imag = np.angle(np.imag(fft_img))  # Phase part
        fft_image = np.stack([fft_real, fft_imag], axis=-1)  # Shape: (28, 28, 2)
        fft_images.append(fft_image)
    return np.array(fft_images)

# ------------- 2. Compute Wavelet Features (4 Channels) -------------
def compute_wavelet_image(images, wavelet='haar', level=1):
    wavelet_images = np.empty((images.shape[0], 14, 14, 4), dtype=np.float16)  # Pre-allocate
    for i, img in enumerate(images):
        coeffs2 = pywt.wavedec2(img[..., 0], wavelet, level=level)  # Wavelet on single channel
        cA, (cH, cV, cD) = coeffs2  # Extract coefficients
        cA, cH, cV, cD = [cv2.resize(c, (14, 14), interpolation=cv2.INTER_LINEAR) for c in [cA, cH, cV, cD]]
        wavelet_images[i] = np.stack([cA, cH, cV, cD], axis=-1)
    return wavelet_images

# ------------- 3. Define CNN Model -------------
def build_input_transformer(input_shape, target_shape=(224, 224, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Resizing(target_shape[0], target_shape[1])(inputs)
    if input_shape[-1] != target_shape[-1]:
        x = Conv2D(target_shape[-1], (3, 3), padding='same', activation='relu')(x)
    model = tf.keras.Model(inputs=inputs, outputs=x, name='input_transformer')
    return model
    
def build_resnet_model(input_shape, num_classes=10, target_shape=(224, 224, 3)):
    transformer = build_input_transformer(input_shape, target_shape)
    inputs = transformer.input
    x = transformer.output
    x = tf.keras.layers.Lambda(lambda img: preprocess_input(img * 255.0))(x)
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=target_shape)
    base_model.trainable = False  # Freeze ResNet layers
    x = base_model(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='resnet_model')
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ModelCheckpoint('vgg16_fashion_mnist_best_model.keras', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

# Model Evaluation Function
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test).argmax(axis=1)
    accuracy = np.mean(y_pred == y_test)
    logging.info(f"{model_name} Test Accuracy: {accuracy:.4f}")
    print(f"=== Classification Report for {model_name} ===")
    print(classification_report(y_test, y_pred))
    print(f"=== Confusion Matrix for {model_name} ===")
    print(confusion_matrix(y_test, y_pred))

# Train CNN on Original MNIST
cnn_mnist = build_resnet_model(input_shape=(28, 28, 1))
cnn_mnist.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_mnist.save("resnet_fashion_mnist_model.keras")
evaluate_model(cnn_mnist, X_test, y_test, "resnet Original MNIST")

# Train CNN on FFT Transformed Images
X_train_fft = compute_fft_image(X_train)
X_test_fft = compute_fft_image(X_test)
cnn_fft = build_resnet_model(input_shape=(28, 28, 2))
cnn_fft.fit(X_train_fft, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_fft.save("resnet_fft_fashion_mnist_model.keras")
evaluate_model(cnn_fft, X_test_fft, y_test, "resnet FFT MNIST")

# Train CNN on Wavelet Transformed Images
X_train_wavelet = compute_wavelet_image(X_train)
X_test_wavelet = compute_wavelet_image(X_test)
cnn_wavelet = build_resnet_model(input_shape=(14, 14, 4))
cnn_wavelet.fit(X_train_wavelet, y_train, epochs=100, validation_split=0.2, callbacks=callbacks, verbose=1)
cnn_wavelet.save("resnet_wavelet_fashion_mnist_model.keras")
evaluate_model(cnn_wavelet, X_test_wavelet, y_test, "resnet Wavelet MNIST")
